# Import packages
run the following cell to import necessary packages to run the analysis

In [1]:
import os
from pathlib import Path

from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
import numpy as np
# from pylibCZIrw import czi as pyczi
# from bioio import BioImage
# import bioio_czi

# Reading data

In [2]:
# data_path = Path('/mnt/Data/nuno_martins/Image_data/Ex011_Re01/ExM/sample_data/')
data_path = Path('/mnt/d/NPMartins/test_data/')
filelist = os.listdir(data_path)
print(len(filelist))

4


In [3]:
filelist = [x for x in filelist if x.find('.tif')>0]
filelist.sort()
filelist

['NPM_Ex011_Re01_a2_postExM_Im-01_AcquisitionBlock4-AiryscanProcessing_sample1.tif',
 'NPM_Ex011_Re01_a2_postExM_Im-01_AcquisitionBlock4-AiryscanProcessing_sample2.tif']

In [4]:
file_index = 0
file_path = str(data_path)+'/'+filelist[file_index]
filename = filelist[file_index][:filelist[file_index].index('.tif')]
print(file_path)
print(filename)

/mnt/d/NPMartins/test_data/NPM_Ex011_Re01_a2_postExM_Im-01_AcquisitionBlock4-AiryscanProcessing_sample1.tif
NPM_Ex011_Re01_a2_postExM_Im-01_AcquisitionBlock4-AiryscanProcessing_sample1


In [5]:
save_path = Path(str(data_path)+'/zarr_export/')
save_path.mkdir(parents=True, exist_ok=True)
print(save_path)

/mnt/d/NPMartins/test_data/zarr_export


___
# Read raw data
reading raw tile data from `czi` tile without stitching for visualization.

In [6]:
from tifffile import imread

In [7]:
image_data = imread(str(data_path) + '/' + filelist[file_index])

In [8]:
image_data.shape

(176, 2, 1021, 1942)

In [11]:
from bioio import BioImage
# import bioio_czi
from bioio.writers.ome_zarr_writer_2 import OmeZarrWriter, compute_level_shapes, compute_level_chunk_sizes_zslice
# from bioio.writers.ome_zarr_writer import OmeZarrWriter

In [12]:
image_shape = (1, image_data.shape[1], image_data.shape[0], image_data.shape[2], image_data.shape[3])
image_shape

(1, 2, 176, 1021, 1942)

In [13]:
scaling = (1, 1, 2, 2, 2)
num_levels = 3

In [14]:
shapes = compute_level_shapes(image_shape, scaling, num_levels)
shapes

[(1, 2, 176, 1021, 1942), (1, 2, 88, 510, 971), (1, 2, 44, 255, 485)]

In [15]:
chunks = compute_level_chunk_sizes_zslice(shapes)
chunks

[(1, 1, 1, 1021, 1942), (1, 1, 4, 510, 971), (1, 1, 16, 255, 485)]

In [16]:
# writer = OmeZarrWriter(str(save_path)+'/sample1.ome.zarr')
writer = OmeZarrWriter()
writer.init_store(str(save_path)+'/sample1_bioio.zarr', shapes, chunks, image_data.dtype)

In [9]:
image_data_save = np.moveaxis(image_data, 0, 1)
image_data_save = np.expand_dims(image_data_save, 0)
image_data_save.shape

(1, 2, 176, 1021, 1942)

In [18]:
writer.write_t_batches_array(image_data_save, tbatch=4)
# writer.write_image(
#     image_data_save,
#     image_name='sample1.ome.zarr',
#     physical_pixel_sizes=None,
#     channel_names=['Cy5', 'DAPI'],
#     # channel_colors=['FF0000', '0080ff'],
#     channel_colors=[255, 125],
# )

In [18]:
# meta = writer.build_ome(
#     size_z=image_shape[2],
#     image_name='sample1.ome.zarr',
#     channel_names=['Cy5', 'DAPI'],
#     channel_colors=[255, 125],
#     channel_minmax=[(0, 3000), (0, 3000)]
# )

In [19]:
meta = writer.generate_metadata(
    image_name='sample1_bioio.zarr',
    channel_names=['Cy5', 'DAPI'],
    physical_dims={"t": 1, "z":0.4, "y":0.047, "x":0.047},
    physical_units={"t":"minute", "z":"micrometer", "y":"micrometer", "x":"micrometer"},
    # channel_colors=["FF0000", "0080ff"],
    # channel_colors=['red', 'cyan'],
    channel_colors=[255, 255],
    # channels_minmax=[(3000, 65535, 0, 0), (3000, 65535, 0, 0)],
)

In [24]:
from bioio.writers.ome_zarr_writer_2 import build_ome

In [20]:
writer.write_metadata(meta)

___
with `ome-zarr`


In [10]:
import zarr
from ome_zarr.io import parse_url
from ome_zarr.writer import write_image

In [ ]:
save_path_2 = str(save_path) + '/' + "sample1_omezarrpy.zarr"

In [23]:
store = parse_url(save_path_2, mode='w').store

In [24]:
root = zarr.group(store=store)

In [25]:
write_image(
    image = image_data_save,
    group=root,
    axes='tczyx',
    # storage_options=dict(chunks=(1, 1, 128, 128))
)

[]

___
save in version 0.3

In [23]:
from ome_zarr.format import FormatV03

In [17]:
save_path_3 = str(save_path) + '/' + "sample1_omezarrpyv0_3.zarr"

In [18]:
store = parse_url(save_path_3, mode='w').store

In [19]:
root = zarr.group(store=store)

In [24]:
write_image(
    image = image_data_save,
    group=root,
    axes='tczyx',
    fmt=FormatV03,
    # storage_options=dict(chunks=(1, 1, 128, 128))
)

TypeError: FormatV01.generate_coordinate_transformations() missing 1 required positional argument: 'shapes'